# **시계열 데이터 처리**


## **1.환경준비**

### **(1) 라이브러리 불러오기**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### **(2) 데이터 준비**

* 원본데이터 불러오기

In [2]:
sales = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_sales_simple.csv")
products = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_product_master.csv")

* 다룰 데이터 만들기
    * 아래 코드를 이해하지 못해도 상관 없습니다.
    * 그냥 실행해주세요.

In [3]:
# 판매액 계산하기
temp = pd.merge(sales, products)
temp['Amt' ] = temp['Qty'] * temp['Price']
temp['Amt'] = (temp['Amt']/1000).round()  # 단위 1000달러

# 집계
data1 = temp.groupby(['Date', 'Category'], as_index = False)['Amt'].sum()
data2 = temp.groupby(['Date'], as_index = False)['Amt'].sum()

# 피봇
data11 = data1.pivot(index='Date', columns='Category', values='Amt').reset_index()

# 합치기
data = pd.merge(data2, data11)

* 우리가 다룰 데이터

In [4]:
data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods
0,2013-01-01,20.0,7.0,4.0,6.0,3.0
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0


## **2.날짜 요소 추출**

### **(1) 날짜 타입으로 변환**
* pd.to_datetime(날짜데이터, format = '입력되는 날짜 형식')
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

In [5]:
data.info()
# Date의 Dtype: object -> String ???

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             31 non-null     object 
 1   Amt              31 non-null     float64
 2   Drink            31 non-null     float64
 3   Food             31 non-null     float64
 4   Grocery          31 non-null     float64
 5   Household Goods  31 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.6+ KB


In [6]:
data['Date'] = pd.to_datetime(data['Date'])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             31 non-null     datetime64[ns]
 1   Amt              31 non-null     float64       
 2   Drink            31 non-null     float64       
 3   Food             31 non-null     float64       
 4   Grocery          31 non-null     float64       
 5   Household Goods  31 non-null     float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 1.6 KB


* format = ''
    * pd.to_datetime(date, format = '%d/%m/%Y') # format = '%d/%m/%Y' 입력되는 날짜가 이런 형태야~~ 라고 알려주는 옵션
    * https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [8]:
# pd.to_datetime(date, format = '%d/%m/%Y') # format = '%d/%m/%Y' 입력되는 날짜가 이런 형태야~~ 라고 알려주는 옵션
date = pd.Series(['03-01-2023', '03-02-2023', '03-03-2023'])
date = pd.to_datetime(date)
date

0   2023-03-01
1   2023-03-02
2   2023-03-03
dtype: datetime64[ns]

In [9]:
# 만약 dd-mm-yyyy 이 맞다면!
date = pd.Series(['03-01-2023', '03-02-2023', '03-03-2023'])
date = pd.to_datetime(date, format = '%d-%m-%Y') #입력받은 날짜 데이터 형식이 '%d-%m-%Y'!
date

0   2023-01-03
1   2023-02-03
2   2023-03-03
dtype: datetime64[ns]

### **(2) 날짜 요소 추출**

In [10]:
# 연도
date.dt.year

0    2023
1    2023
2    2023
dtype: int32

In [11]:
# 월
date.dt.month

0    1
1    2
2    3
dtype: int32

In [59]:
# 일
date.dt.day

0    3
1    3
2    3
dtype: int32

In [12]:
# 요일
# 0: 월
date.dt.weekday

0    1
1    4
2    4
dtype: int32

In [13]:
# 요일 이름
date.dt.day_name()

0    Tuesday
1     Friday
2     Friday
dtype: object

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] data의 Date는 이미 날짜 타입으로 변환되어 있습니다.  
* 다음의 항목을 열로 추가하시오.
    * 요일(이름)
    * 주차

In [14]:
data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods
0,2013-01-01,20.0,7.0,4.0,6.0,3.0
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0


In [15]:
data['WeekDay'] = data['Date'].dt.day_name()
data['Week'] = data['Date'].dt.isocalendar().week
data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods,WeekDay,Week
0,2013-01-01,20.0,7.0,4.0,6.0,3.0,Tuesday,1
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0,Wednesday,1
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0,Thursday,1
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0,Friday,1
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0,Saturday,1


## **3.시간에 따른 흐름 추가하기**

### **(1) shift**
* 시계열 데이터에서 시간의 흐름 전후로 정보를 이동시킬 때 사용
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html

In [64]:
temp = data.loc[:,['Date','Amt']]

In [16]:
# 전날 매출액 열을 추가합시다.
temp['Amt_lag'] = temp['Amt'].shift() #default = 1

# 전전날 매출액 열을 추가.
temp['Amt_lag2'] = temp['Amt'].shift(2) # 2행 shift

# 다음날 매출액 열을 추가합시다.
temp['Amt_lag_1'] = temp['Amt'].shift(-1)

temp.head()

,Order_ID,Date,Store_ID,Qty,Product_ID,Product_Code,SubCategory,Category,LeadTime,Price,Amt,Amt_lag,Amt_lag2,Amt_lag_1
0,3,2013-01-01,1,0.0,3,DB001,Beverage,Drink,2,8,0.0,NaN,NaN,0.0
1,36,2013-01-01,10,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,NaN,0.0
2,69,2013-01-01,11,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0
3,102,2013-01-01,12,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0
4,135,2013-01-01,13,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0


### **(2) rolling + 집계함수**
* 시간의 흐름에 따라 일정 기간 동안 평균을 이동하면서 구하기
* .rolling : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
* rolling(n) : 
    * n 기본값은 1
    * min_periods : 최소 데이터수

In [17]:
# 7일 이동평균 매출액을 구해 봅시다.
temp['Amt_MA7_1'] = temp['Amt'].rolling(7).mean()
temp['Amt_MA7_2'] = temp['Amt'].rolling(7, min_periods = 1).mean()
temp.head(10)

,Order_ID,Date,Store_ID,Qty,Product_ID,Product_Code,SubCategory,Category,LeadTime,Price,Amt,Amt_lag,Amt_lag2,Amt_lag_1,Amt_MA7_1,Amt_MA7_2
0,3,2013-01-01,1,0.0,3,DB001,Beverage,Drink,2,8,0.0,NaN,NaN,0.0,NaN,0.0
1,36,2013-01-01,10,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,NaN,0.0,NaN,0.0
2,69,2013-01-01,11,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0,NaN,0.0
3,102,2013-01-01,12,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0,NaN,0.0
4,135,2013-01-01,13,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0,NaN,0.0
5,168,2013-01-01,14,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0,NaN,0.0
6,201,2013-01-01,15,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0,0.0,0.0
7,234,2013-01-01,16,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0,0.0,0.0
8,267,2013-01-01,17,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0,0.0,0.0
9,300,2013-01-01,18,0.0,3,DB001,Beverage,Drink,2,8,0.0,0.0,0.0,0.0,0.0,0.0


### (3) **diff**
* 특정 시점 데이터, 이전시점 데이터와의 차이 구하기
* .diff : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] data의 Grocery가 매출이 가장 높습니다. 이에 대해서 다음의 열을 추가해 봅시다.

* 전날 매출액
* 7일 전(전주 동 요일) 매출액
* 3일 이동평균 매출액
* 전날대비 매출액 증감여부 (증가 1, 감소 -1, 동일 0)


In [18]:
data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods,WeekDay,Week
0,2013-01-01,20.0,7.0,4.0,6.0,3.0,Tuesday,1
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0,Wednesday,1
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0,Thursday,1
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0,Friday,1
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0,Saturday,1


In [19]:
tmp = data.loc[:, ['Date', 'Grocery']]
tmp.head()

,Date,Grocery
0,2013-01-01,6.0
1,2013-01-02,1663.0
2,2013-01-03,1222.0
3,2013-01-04,1252.0
4,2013-01-05,1560.0


In [23]:
# 전날 매출액
tmp['Gro_lag'] = tmp['Grocery'].shift()

# 7일 전(전주 동 요일) 매출액
tmp['Gro_lag7'] = tmp['Grocery'].shift(7)

# 3일 이동평균 매출액
tmp['MA3'] = tmp['Grocery'].rolling(3, min_periods=1).mean()

# 전날대비 매출액 증감여부 (증가 1, 감소 -1, 동일 0)
tmp['Diff'] = np.where(tmp['Grocery'].diff() > 0,  1,
                      np.where(tmp['Grocery'].diff() == 0, 0, -1))
tmp.head()

,Date,Grocery,Gro_lag,Gro_lag7,MA3,Diff
0,2013-01-01,6.0,NaN,NaN,6.000000,-1
1,2013-01-02,1663.0,6.0,NaN,834.500000,1
2,2013-01-03,1222.0,1663.0,NaN,963.666667,-1
3,2013-01-04,1252.0,1222.0,NaN,1379.000000,1
4,2013-01-05,1560.0,1252.0,NaN,1344.666667,1
